In [4]:
!pip3 install mlflow

##               **** Training the Heart-attack Prediction model ****

### --> Importing Libraries

In [5]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pickle
from sklearn.metrics import f1_score ,accuracy_score

### --> Loading Data

In [6]:
main_df = pd.read_csv("heart.csv")
df = main_df.copy()


### --> Data Preprocessing

In [7]:
cols = ["trtbps", "chol", "thalachh", "oldpeak", "age"]
for col in cols:
    minimum = min(df[col])
    maximum = max(df[col])
    df[col] = (df[col] - minimum) / (maximum - minimum)

In [8]:
X = df.drop("output", axis=1)
y = df["output"]

### --> Spliting Training and testing data

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

### --> Training model

In [10]:
model_lg = LogisticRegression(max_iter=120, random_state=0, n_jobs=20)
model_lg.fit(X_train, y_train)

LogisticRegression(max_iter=120, n_jobs=20, random_state=0)

### --> Calculating the metrics

In [11]:
y_pred = model_lg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

### --> Saving the model

In [12]:
filename = "heart-attack-prediction.pkl"
pickle.dump(model_lg, open(filename, "wb"))
print("Model File Created...")

Model File Created...


##               **** Logging the Heart-attack Prediction model ****

### --> Importing Libraries

In [13]:
import pickle
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from mlflow.types.schema import Schema, ColSpec
from mlflow.models.signature import ModelSignature
from mlflow import log_metrics

### --> Loading Model

In [14]:
infile = open("heart-attack-prediction.pkl", "rb")
model = pickle.load(infile)
print("Loaded Model")
infile.close()

Loaded Model


### --> Creating Signature

In [15]:
mlflow.set_tracking_uri("sqlite:///mlruns.db")

input_schema = Schema(
    [
        ColSpec("double", "age"),
        ColSpec("integer", "sex"),
        ColSpec("integer", "cp"),
        ColSpec("double", "trtbps"),
        ColSpec("double", "chol"),
        ColSpec("integer", "fbs"),
        ColSpec("integer", "restecg"),
        ColSpec("double", "thalachh"),
        ColSpec("integer", "exng"),
        ColSpec("double", "oldpeak"),
        ColSpec("integer", "slp"),
        ColSpec("integer", "caa"),
        ColSpec("integer", "thall"),
    ]
)

signature = ModelSignature(inputs=input_schema)

### --> Logging Model artifacts

In [16]:
mlflow.sklearn.log_model(
    model,
    "heart-attack-prediction-model",
    registered_model_name="heart-attack-prediction-model",
    signature=signature,
)
print("Logged another model")

/opt/python/3.10.4/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'heart-attack-prediction-model' already exists. Creating a new version of this model...
2022/06/19 09:54:42 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: heart-attack-prediction-model, version 2


Logged another model


Created version '2' of model 'heart-attack-prediction-model'.


### --> Logging Metric

In [19]:
metrics = {"accuracy": accuracy, "f1_score": f1}
mlflow.log_metrics(metrics)

### --> Transition into production

In [21]:
client = MlflowClient()
client.transition_model_version_stage(
    name="heart-attack-prediction-model", version=1, stage="Production"
)

<ModelVersion: creation_timestamp=1655631152321, current_stage='Production', description=None, last_updated_timestamp=1655632548246, name='heart-attack-prediction-model', run_id='4f238b29f4dd4f308f62c160a49d4190', run_link=None, source='./mlruns/0/4f238b29f4dd4f308f62c160a49d4190/artifacts/heart-attack-prediction-model', status='READY', status_message=None, tags={}, user_id=None, version=1>